## DNN

In [2]:
from epftoolbox.models import hyperparameter_optimizer


In [25]:
%run C:\Users\CTLGNN00C\Documents\GitHub\epftoolbox\epftoolbox\data\_datasets.py
%run C:\Users\CTLGNN00C\Documents\GitHub\epftoolbox\epftoolbox\models\_dnn.py

In [4]:
path = "C:/Users/CTLGNN00C/Documents/ENERGY/Progetto MGP"

In [5]:
train_df, test_df = read_data(path, dataset = "FR") # Grouped dataframes (by date)

### Hyperparameters optimizer (don't run again)

In [8]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = 'FR'

# Number of years (a year is 364 days) in the test dataset.
years_test = 2

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = None
end_test_date = None

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 4

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 1

# Number of iterations for hyperparameter optimization
max_evals = 10

path_datasets_folder = path
path_hyperparameters_folder = path +  "/experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

In [9]:
import pickle as pc

# Percorso dove è stato salvato il file di ottimizzazione
trials_file_path = os.path.join(path_hyperparameters_folder, 'DNN_hyperparameters_nl' + str(nlayers) +
                                '_dat' + str(dataset) + '_YT' + str(years_test) + 
                                '_SF' * shuffle_train + '_DA' * data_augmentation + 
                                '_CW' + str(calibration_window) + '_' + str(experiment_id))

# Caricare il file di trials salvato
with open(trials_file_path, "rb") as f:
    trials = pc.load(f)


In [10]:
# Miglior risultato dato dall'ottimizzatore
trials.best_trial

{'state': 2,
 'tid': 5,
 'spec': None,
 'result': {'loss': 6.541263089302935,
  'MAE Val': 6.541263089302935,
  'MAE Test': 7.3781093193566525,
  'sMAPE Val': 14.114707598008739,
  'sMAPE Test': 20.11082947304353,
  'status': 'ok'},
 'misc': {'tid': 5,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'In: Day': [5],
   'In: Exog-1 D': [5],
   'In: Exog-1 D-1': [5],
   'In: Exog-1 D-7': [5],
   'In: Exog-2 D': [5],
   'In: Exog-2 D-1': [5],
   'In: Exog-2 D-7': [5],
   'In: Price D-1': [5],
   'In: Price D-2': [5],
   'In: Price D-3': [5],
   'In: Price D-7': [5],
   'activation': [5],
   'batch_normalization': [5],
   'dropout': [5],
   'init': [5],
   'lambdal1': [],
   'lr': [5],
   'neurons1': [5],
   'neurons2': [5],
   'reg': [5],
   'scaleX': [5],
   'scaleY': [5],
   'seed': [5]},
  'vals': {'In: Day': [1],
   'In: Exog-1 D': [1],
   'In: Exog-1 D-1': [1],
   'In: Exog-1 D-7': [1],
   'In: Exog-2 D': [1],
   'In: Exog-2 D-1': [0],
   'In: Exog-2 D

In [11]:
# Parametri migliori per la migliore configurazione
best_trial = trials.best_trial
best_hyperparams = best_trial['misc']['vals']
best_hyperparams

{'In: Day': [1],
 'In: Exog-1 D': [1],
 'In: Exog-1 D-1': [1],
 'In: Exog-1 D-7': [1],
 'In: Exog-2 D': [1],
 'In: Exog-2 D-1': [0],
 'In: Exog-2 D-7': [0],
 'In: Price D-1': [0],
 'In: Price D-2': [0],
 'In: Price D-3': [1],
 'In: Price D-7': [0],
 'activation': [5],
 'batch_normalization': [1],
 'dropout': [0.438890096383171],
 'init': [1],
 'lambdal1': [],
 'lr': [0.003803651200105018],
 'neurons1': [359.0],
 'neurons2': [101.0],
 'reg': [0],
 'scaleX': [3],
 'scaleY': [4],
 'seed': [318.0]}

In [12]:
# Definizione iperparametri
activation = best_hyperparams['activation'][0]
batch_normalization = best_hyperparams['batch_normalization'][0]
dropout = best_hyperparams['dropout'][0]
learning_rate = best_hyperparams['lr'][0]
neurons1 = int(best_hyperparams['neurons1'][0])  # Converti in int per i neuroni
neurons2 = int(best_hyperparams['neurons2'][0])
scaleX = best_hyperparams['scaleX'][0]
scaleY = best_hyperparams['scaleY'][0]
seed = int(best_hyperparams['seed'][0])

In [45]:
# Definizione features
features = {}
for feat in list(best_hyperparams.keys())[0:11]:
    features[feat] = best_hyperparams[feat][0]


In [13]:
opt_hyperparams = format_best_trial(best_trial)

In [32]:
train_df

,Price,Exogenous 1,Exogenous 2
Date,,,
2011-01-09 00:00:00,32.542,63065.0,63000.0
2011-01-09 01:00:00,21.549,62715.0,58800.0
2011-01-09 02:00:00,15.711,61952.0,58500.0
2011-01-09 03:00:00,10.583,59262.0,54300.0
2011-01-09 04:00:00,10.324,56883.0,51900.0
...,...,...,...
2015-01-03 19:00:00,47.620,77508.0,68739.0
2015-01-03 20:00:00,43.790,75506.0,66734.0
2015-01-03 21:00:00,42.440,72883.0,64515.0


In [35]:
_build_and_split_XYs(dfTrain=train_df,dfTest=test_df,  features=opt_hyperparams, n_exogenous_inputs=2, shuffle_train=True, percentage_val=0.25,
                        date_test=None, hyperoptimization=False, data_augmentation=False)

IndexError: index 97 is out of bounds for axis 1 with size 97

In [26]:
opt_hyperparams

{'In: Day': 1,
 'In: Exog-1 D': 1,
 'In: Exog-1 D-1': 1,
 'In: Exog-1 D-7': 1,
 'In: Exog-2 D': 1,
 'In: Exog-2 D-1': 0,
 'In: Exog-2 D-7': 0,
 'In: Price D-1': 0,
 'In: Price D-2': 0,
 'In: Price D-3': 1,
 'In: Price D-7': 0,
 'activation': 'PReLU',
 'batch_normalization': 1,
 'dropout': 0.438890096383171,
 'init': 'lecun_uniform',
 'lambdal1': 0,
 'lr': 0.003803651200105018,
 'neurons1': 359.0,
 'neurons2': 101.0,
 'reg': None,
 'scaleX': 'Std',
 'scaleY': 'Median',
 'seed': 318.0}